In [ ]:
import ast
import bs4
import time
import json
import requests
import pandas as pd
import scipy as scipy
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
import matplotlib.pyplot as plt

from multiprocessing.pool import ThreadPool




%matplotlib inline

In [51]:

def get_strains_w_flavors(page_num):
    
    strain_urls = []

    url = f'https://www.strainofweed.com/page/{i}/'
    r = requests.get(url)
    soup = BeautifulSoup(r.content,'html.parser')
    names = soup.find_all('a',{'rel':'bookmark'})
    for name in names:
        strain_urls.append(name['href'])

    return strain_urls


In [ ]:
url = 'https://www.strainofweed.com/hybrid/gsc/'

r = requests.get(url)

soup = BeautifulSoup(r.content,'html.parser')

flavors = soup.find_all('td')[9]
name = soup.find_all('td')[1]


In [52]:

def scrape_flavors(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content,'html.parser')
    flavors = soup.find_all('td')[9].text.split(",")
    flavor_list = [x.replace(' ','') for x in flavors]
    name = soup.find_all('td')[1].text


    return name,flavor_list





In [53]:
def run_flavors_thread(mode):

    if mode == 'Flavors':
    
        flavor_data = []

        with ThreadPool(10) as pool:
            for name,flavor_list in pool.map(scrape_flavors,strain_urls):
                flavor_data.append({'Strain Name':name,'Flavors':flavor_list})

        return flavor_data
    
    if mode == 'Names':

        names = []
        page_num = list(range(1,180))
        with ThreadPool(10) as pool:
            for result in pool.map(get_strains_w_flavors,page_num):
                names.append(result)

        flat_names = [item for sublist in names for item in sublist]

        return flat_names


In [55]:
start = time.perf_counter()
flavor_data = run_flavors_thread('Names')
finish = time.perf_counter()
print(f'Finished in {round((finish-start)/60,2)} minutes')

Finished in 1.16 minutes


In [ ]:
# start = time.perf_counter()
# flavor_data = run_thread()
# finish = time.perf_counter()
# print(f'Finished in {round((finish-start)/60,2)} minutes')

In [ ]:
print(flavor_data[6])


with open('flavors.txt','w') as f:
    json.dump(flavor_data,f)

In [ ]:
with open('flavors.txt', 'r') as f:
    flavor_data = json.load(f)


In [ ]:
flavor_strain_names = [x['Strain Name'] for x in flavor_data]